Fazer as transformações que serão necessário fazer na parte de ETL do banco de dados que passa pelo airflow e é carregado no nosso banco de dados.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
dados_clientes = r"C:\Users\heloi\OneDrive\Desktop\Projeto - Inlytics\Dataset\clientes_info_limpo.csv"
dados_vendas = r"C:\Users\heloi\OneDrive\Desktop\Projeto - Inlytics\Dataset\vendas_info_limpo.csv"

df_clientes = pd.read_csv(dados_clientes)
df_vendas = pd.read_csv(dados_vendas)

In [4]:
df_clientes.head()

,CustomerID,Churn,Tenure,PreferredLoginDevice,PreferredPaymentMode,Gender,NumberOfDeviceRegistered,SatisfactionScore,MaritalStatus,Complain,CouponUsed,NomeCustomer
0,17850.0,1,4.0,Mobile Phone,Debit Card,Female,3.0,2,Single,1,1.0,Miguel Brito
1,13047.0,1,NaN,Phone,UPI,Male,4.0,3,Single,1,0.0,Sr. Ryan Santos
2,12583.0,1,NaN,Phone,Debit Card,Male,4.0,3,Single,1,0.0,Enzo Gabriel Abreu
3,13748.0,1,0.0,Phone,Debit Card,Male,4.0,589314,Single,0,0.0,Sra. Eloá Campos
4,15100.0,1,0.0,Phone,CC,Male,3.0,5,Single,0,1.0,Camila Teixeira


In [5]:
df_vendas.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Limpeza do 'Tenure'

- Dados inconsistentes
- Dados nulos

Dados que estão inconsistentes, pois no tenure (quantidade de tempo dos clientes na empresa) não faz sentido ter dados negarivos, somente dados positivos. Com isso para não perder dados estarei colocando-os para 0.

In [6]:
df_clientes.loc[df_clientes['Tenure'] <= 0, 'Tenure'] = 0
df_clientes.loc[df_clientes['Tenure'].isnull(), 'Tenure'] = 0


In [7]:
df_clientes.isnull().sum()

CustomerID                    1
Churn                         0
Tenure                        0
PreferredLoginDevice          0
PreferredPaymentMode        154
Gender                        0
NumberOfDeviceRegistered    154
SatisfactionScore             0
MaritalStatus               165
Complain                      0
CouponUsed                  199
NomeCustomer                  0
dtype: int64

In [8]:
df_vendas.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [9]:
df_vendas.shape

(397884, 8)

In [10]:
df_clientes.shape

(4373, 12)

In [20]:
import sqlite3
conn = sqlite3.connect(r"C:\Users\heloi\OneDrive\Desktop\Projeto - Inlytics\Dataset\db\empresa_cliente_db.db")
sales = pd.read_sql_query("SELECT * FROM customer_sales", conn)
user = pd.read_sql_query("SELECT * FROM customer_user", conn)

user.columns = user.iloc[0]
user = user[1:].reset_index(drop=True)

sales.columns = sales.iloc[0]
sales = sales[1:].reset_index(drop=True)


In [24]:
user.head()

,CustomerID,Churn,Tenure,PreferredLoginDevice,PreferredPaymentMode,Gender,NumberOfDeviceRegistered,SatisfactionScore,MaritalStatus,Complain,CouponUsed,NomeCustomer
0,17850,1,4.0,Mobile Phone,Debit Card,Female,3.0,2,Single,1,1.0,Dr. Thales Gonçalves
1,13047,1,,Phone,UPI,Male,4.0,3,Single,1,0.0,Dra. Letícia Sá
2,12583,1,,Phone,Debit Card,Male,4.0,3,Single,1,0.0,Dr. Murilo Mendes
3,13748,1,0.0,Phone,Debit Card,Male,4.0,589314,Single,0,0.0,Sra. Allana Moura
4,15100,1,0.0,Phone,CC,Male,3.0,5,Single,0,1.0,Sarah Jesus


In [27]:
user[user['CustomerID'].isnull()]

,CustomerID,Churn,Tenure,PreferredLoginDevice,PreferredPaymentMode,Gender,NumberOfDeviceRegistered,SatisfactionScore,MaritalStatus,Complain,CouponUsed,NomeCustomer


In [ ]:
conn.close()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customerID,country
0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
2,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
3,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
4,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
